In [16]:
import requests
import pandas as pd
from time import sleep
import ast

# Alpha Vantage API Key https://www.alphavantage.co/support/#api-key
API_KEY = ""


equities = ['XOM','CVX', 'SHEL', 'COP', 'BP', 'PBR']
more_equities = ['WTI']

crude_oil = []#['CL=F', 'BZ=F'] # wti, brent, 

tickers = equities + more_equities + crude_oil


def to_df(decoded_content, ticker):
    """CSV string to df"""
    lines = decoded_content.splitlines()
    df = pd.DataFrame([row.split(',') for row in lines[1:]], 
                    columns=["date", *lines[0].split(',')[1:]])

    df = df.reset_index(drop=True).set_index('date')
    df = pd.concat([df], axis=1, keys=[ticker])
    return df
    


In [ ]:
# ticker = "XOM"
# year = 1
# month = 1
# # get_ticker_data("CVX")

# with requests.Session() as s:
#     download = s.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol={'CVX'}&interval={'1min'}&slice=year{'1'}month{'1'}&apikey={API_KEY}")
#     decoded_content = download.content.decode('utf-8')

#     print(decoded_content)

# with requests.Session() as s:
#     download = s.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={'CVX'}&interval={'15min'}&apikey={API_KEY}")
#     decoded_content = download.content.decode('utf-8')

#     print(decoded_content)

# data["CVX"]

##### Get Data From Data Provider

In [20]:
def get_ticker_data(ticker, time="1min", year=1, month=1):
    """Function to get (ticker, year, month) data using alpha vantage's time series intraday extended API"""
    CSV_URL = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol={ticker}&interval={time}&slice=year{year}month{month}&apikey={API_KEY}"

    

    while True:
        with requests.Session() as s:
            download = s.get(CSV_URL)
            decoded_content = download.content.decode('utf-8')
            print(f"ticker: {ticker}, y{year} m{month}; response length: {len(decoded_content)}")
            if len(decoded_content) == 236:
                # API too many requests
                sleep(60)
            elif len(decoded_content) < 236:
                # Token doesn't exist or something
                print(f"Error getting {ticker}, y{year}, m{month}. We are skipping")
                return None
            else:
                return to_df(decoded_content, ticker)


def use_alpha_vantage(tickers, time= "1min", out_file="realdata.csv"):
    """Function to get multiple full tickers data using alpha vantage's time series intraday extended API"""
    dfs = []
    for ticker in tickers:
        t_dfs = []
        for year in range(1,3):
            for month in range(1,13):
                df_temp = get_ticker_data(ticker, time=time, year=year, month=month)
                if df_temp is not None:
                    t_dfs.append(df_temp)

        if len(t_dfs):
            dfs.append(pd.concat(t_dfs, axis=0))
        else:
            print(f"Skipped {ticker}.")
    df = pd.concat(dfs, axis=1, sort=True)
    df.index.rename('date', inplace=True)
    df.to_csv(out_file)
    return df

df = use_alpha_vantage(tickers, out_file="realdata.csv")

ticker: XOM y: 1 m: 1; response length 609011
ticker: XOM y: 1 m: 2; response length 567616
ticker: XOM y: 1 m: 3; response length 579176
ticker: XOM y: 1 m: 4; response length 961371
ticker: XOM y: 1 m: 5; response length 837093
ticker: XOM y: 1 m: 6; response length 236


KeyboardInterrupt: 

##### Read Data From All-Data CSV (Multi Index Columns)

In [4]:
def read_data(out_file="realdata.csv"):
    data = pd.read_csv(out_file, index_col=0)
    data.columns = data.columns.map(ast.literal_eval)
    return data

df_all = read_data("realdata.csv")
df_all.head()

XOM                                          CVX  \
                          open       high        low      close volume open   
date                                                                          
2020-11-18 04:01:00  35.343754  35.471612  35.343754  35.471612  454.0  NaN   
2020-11-18 04:03:00  35.398550  35.398550  35.398550  35.398550  100.0  NaN   
2020-11-18 04:04:00        NaN        NaN        NaN        NaN    NaN  NaN   
2020-11-18 04:07:00  35.371152  35.371152  35.261559  35.261559  400.0  NaN   
2020-11-18 04:09:00        NaN        NaN        NaN        NaN    NaN  NaN   

                                           ...  PBR                        \
                    high low close volume  ... open high low close volume   
date                                       ...                              
2020-11-18 04:01:00  NaN NaN   NaN    NaN  ...  NaN  NaN NaN   NaN    NaN   
2020-11-18 04:03:00  NaN NaN   NaN    NaN  ...  NaN  NaN NaN   NaN    NaN   
2020-11-18 04:04:00  NaN NaN   NaN    NaN  ...  NaN  NaN NaN   NaN    NaN   
2020-11-18 04:07:00  NaN NaN   NaN    NaN  ...  NaN  NaN NaN   NaN    NaN   
2020-11-18 04:09:00  NaN NaN   NaN    NaN  ...  NaN  NaN NaN   NaN    NaN   

                     WTI                        
                    open high low close volume  
date                                            
2020-11-18 04:01:00  NaN  NaN NaN   NaN    NaN  
2020-11-18 04:03:00  NaN  NaN NaN   NaN    NaN  
2020-11-18 04:04:00  NaN  NaN NaN   NaN    NaN  
2020-11-18 04:07:00  NaN  NaN NaN   NaN    NaN  
2020-11-18 04:09:00  NaN  NaN NaN   NaN    NaN  

[5 rows x 35 columns]

##### Filter df_all to just Close data

In [6]:
def just_close_data(data):
    return data.xs("close",level=1, axis=1)

df_close = just_close_data(df_all)
df_close.head()

,XOM,CVX,SHEL,COP,BP,PBR,WTI
date,,,,,,,
2020-11-18 04:01:00,35.471612,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-18 04:03:00,35.398550,NaN,NaN,NaN,18.171303,NaN,NaN
2020-11-18 04:04:00,NaN,NaN,NaN,NaN,18.134481,NaN,NaN
2020-11-18 04:07:00,35.261559,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-18 04:09:00,NaN,NaN,NaN,NaN,18.134481,NaN,NaN


##### Fill NaNs

In [14]:
def ffill_nans(df):
    df = df.fillna(method="ffill")
    df = df.dropna()
    return df

def del_nans_ffill(df, thresh):
    df = df.dropna(thresh=thresh)
    df = ffill_nans(df)
    return df


df_processed_dups = ffill_nans(df_close) #138,607
df_processed_holes = del_nans_ffill(df_close, 3) #91,693
# df_close #348,724


##### Save Data

In [15]:
df_processed_dups.to_csv("data/ETT/processed_dups.csv")
df_processed_holes.to_csv("data/ETT/processed_holes.csv")